In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.api.types import CategoricalDtype

print(pd.__version__)

%matplotlib inline

0.22.0


In [2]:
# Read table

mercari_data = pd.read_table("./train.tsv", index_col = 'train_id', dtype = {'item_condition_id':CategoricalDtype(categories = [str(i) for i in range(1,6)], ordered = True), 'category_name':'category', 'brand_name': 'category', 'shipping':'category'})

/Users/Abe/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:466: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
cat1, cat2, cat3, a = mercari_data.category_name.str.split("/", 3).str
mercari_data['category_1'] = cat1
mercari_data['category_2'] = cat2
mercari_data['category_3'] = cat3
for col in ['category_1', 'category_2', 'category_3']:
    mercari_data[col] = mercari_data[col].astype('category')

In [4]:
mercari_data['log_price'] = np.log(mercari_data.price + 1)

In [24]:
# The goal here is to use comparisons of RSS for different category levels in linear models, balanced with
# the increase in k the number of levels of categoried used.
# The comparisons are based on AIC comparisons and used to determine the optimal number of levels to include
# in the model, but separately for each subcategory
# The goal is to come up with a linear equation that takes a different number of levels into account for each
# subcategory to avoid overfitting

# The model with more predictors is used for a subcategory if and only if
# 2+(n*(ln(RSS(i+1)/RSS(i)))) < 0
# where RSS(i+1) is the residual sum of squares for the model with one more predictor, and RSS(i)
# is the residual sum of squares for the original model

# Testing on a common subcategory, where we decide whether to include "Electronics/Cameras & Photography"

data_subset = mercari_data[(mercari_data['category_1'] == 'Electronics') & (mercari_data['category_2'] == 'Cameras & Photography')]

def rss(sampled, predicted): # Sampled is a series (i.e. a sample), and predicted is a numeric scalar
    return ((sampled - predicted)**2).sum()

print(rss(data_subset['log_price'], data_subset['log_price'].mean()))

supercategory_mean = mercari_data[mercari_data['category_1'] == 'Electronics'].log_price.mean()
print(supercategory_mean)
print(data_subset['log_price'].mean())
print('RSS orig: ' + str(rss(data_subset['log_price'], supercategory_mean)))
print('RSS new: ' + str(rss(data_subset['log_price'], data_subset['log_price'].mean())))
print('Count: ' + str(data_subset.log_price.count()))
print('ln: ' + str(np.log(6166/3891)))
print('n*...: ' + str(3976*np.log(6166/3891)))

3891.3834929348423
2.9862535475404206
3.7427124054842973
RSS orig: 6166.569987891458
RSS new: 3891.3834929348423
Count: 3976
ln: 0.46038413546154205
n*...: 1830.4873225950912
